# Linear Programming Mini Example

Linear programming (linear optimization) is a mathematical technique that uses a set of linear equations to find the optimal solution to a problem. It's an effective type of mathematical programming that's used to solve complex real-world problems, e.g. in energy systems.

**1. Load packages**
   - We are using the gurobipy package to formulate a mathematical model and solve it. 

In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

**2. Define parameters**
   - _G_: Number of generators **2**
   - _$\overline{P}_i$_: Production limit **[6,4]** in MW 
   - _$c_i$_: Cost **[0.015,0.03]** in $/kWh
   - _D_: Load demand **8** in MW 

In [3]:
n_generators = 2 # G
p_lim = np.array([6,4]) # MW
cost = np.array([0.015,0.03]) # $/kWh
D = 8 # MW

**3. Define mathematical model**

Objective function: 
- Minimize the sum cost _$c_i$_, which depends on the production _$p_i$_ amount of each generator _i_.

Decision variables:
- _$p_i$_ output of generator _i_.

Constraints: 
- The sum of generation _$p_i$_ for each generator _i_ needs to equal the load _D_. 
- The output of each generator _$p_i$_ has to greater equal to $0$ and cannot exceed _$\overline{P}_i$_.

\begin{align}
\min \quad 
    & \sum_{i\in[G]}c_i p_i \\
\text{s.t.} \quad 
    & \sum_{i\in[G]}p_i = D \\
    & 0 \le p_i \le \overline{P}_i && \forall i \in [G] 
\end{align}

In [ ]:
# Create model object
m = gp.Model()
m.setParam("OutputFlag", 0)

# Create the variables
p = m.addVars(n_generators, lb=0, ub=GRB.INFINITY, name="p")

# Add constraints:
# Total production
m.addConstr(p.sum() == D)

# Generator limits
for i in range(n_generators):
    m.addConstr(p[i] <= p_lim[i])

# Add objective
m.setObjective(sum(p[i]*cost[i]*1000 for i in range(n_generators)), GRB.MINIMIZE)

# Solve
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-10
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x930ee9ad
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 8e+00]
Presolve removed 3 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.500000000e+02


**4. Inspect the solution**

In [5]:
# Check the status of the solver
status = m.Status
if status == GRB.OPTIMAL:
    print("The model is optimal.")
elif status == GRB.INFEASIBLE:
    print("The model is infeasible.")
elif status == GRB.UNBOUNDED:
    print("The model is unbounded.")
else:
    print(f"Optimization ended with status {status}.")
print()

# Objective value
objective = m.ObjVal
print(f"Objective value {objective:.1f}.\n")

print("Variables values:")
# Print the values of all variables
for v in m.getVars():
    print(f"{v.VarName} = {v.X}")

The model is optimal.

Objective value 150.0.

Variables values:
p[0] = 6.0
p[1] = 2.0
